In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras import layers
import warnings
from utils import clean_columns
warnings.simplefilter('once')

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/remijul/dataset/master/Airline%20Passenger%20Satisfaction.csv', sep=';')
df_clean = clean_columns(df).dropna().drop('id',axis=1)
df_clean.columns

Index(['Satisfaction', 'Gender', 'Customer_Type', 'Age', 'Type_of_Travel',
       'Class', 'Flight_Distance', 'Seat_comfort',
       'Departure_Arrival_time_convenient', 'Food_and_drink', 'Gate_location',
       'Inflight_wifi_service', 'Inflight_entertainment', 'Online_support',
       'Ease_of_Online_booking', 'On_board_service', 'Leg_room_service',
       'Baggage_handling', 'Checkin_service', 'Cleanliness', 'Online_boarding',
       'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes'],
      dtype='object')

In [14]:
[print(col, df[col].unique()) for col in df_clean.columns]

Satisfaction ['satisfied' 'neutral or dissatisfied']
Gender ['Female' 'Male']
Customer_Type ['Loyal Customer' 'disloyal Customer']
Age [65 47 15 60 70 30 66 10 56 22 58 34 62 35 13 52 55 28  9 25 53 16 64 42
 21 20 26 48 57 31 17 33 32 38 29 24 37  7 39 11 49  8 40 45 67 59 44 69
 51 18 23 12 46 41 54 27 63 61 36 50 68 19 14 43 72 71 80 77 85 78 75 79
 74 73 76]
Type_of_Travel ['Personal Travel' 'Business travel']
Class ['Eco' 'Business' 'Eco Plus']
Flight_Distance [ 265 2464 2138 ... 5832 5120 4260]
Seat_comfort [0 1 4 5 2 3]
Departure_Arrival_time_convenient [0 1 2 3 4 5]
Food_and_drink [0 1 2 3 4 5]
Gate_location [2 3 4 1 5 0]
Inflight_wifi_service [2 0 3 4 5 1]
Inflight_entertainment [4 2 0 3 5 1]
Online_support [2 3 4 5 1 0]
Ease_of_Online_booking [3 2 1 5 4 0]
On_board_service [3 4 1 2 5 0]
Leg_room_service [0 4 3 2 5 1]
Baggage_handling [3 4 1 2 5]
Checkin_service [5 2 4 3 1 0]
Cleanliness [3 4 1 2 5 0]
Online_boarding [2 3 5 4 1 0]
Departure_Delay_in_Minutes [   0  310   17   3

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [15]:
df_clean.dtypes

Satisfaction                          object
Gender                                object
Customer_Type                         object
Age                                    int64
Type_of_Travel                        object
Class                                 object
Flight_Distance                        int64
Seat_comfort                           int64
Departure_Arrival_time_convenient      int64
Food_and_drink                         int64
Gate_location                          int64
Inflight_wifi_service                  int64
Inflight_entertainment                 int64
Online_support                         int64
Ease_of_Online_booking                 int64
On_board_service                       int64
Leg_room_service                       int64
Baggage_handling                       int64
Checkin_service                        int64
Cleanliness                            int64
Online_boarding                        int64
Departure_Delay_in_Minutes             int64
Arrival_De

In [16]:
val_dataframe = df_clean.sample(frac=0.2, random_state=1337)
train_dataframe = df_clean.drop(val_dataframe.index)

In [17]:
print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 103590 samples for training and 25897 for validation


In [18]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("Satisfaction")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [19]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

2023-04-06 15:06:50.606300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [103590]
	 [[{{node Placeholder/_4}}]]
2023-04-06 15:06:50.607217: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_21' with dtype string and shape [103590]
	 [[{{node Placeholder/_21}}]]


Input: {'Gender': <tf.Tensor: shape=(), dtype=string, numpy=b'Female'>, 'Customer_Type': <tf.Tensor: shape=(), dtype=string, numpy=b'Loyal Customer'>, 'Age': <tf.Tensor: shape=(), dtype=int64, numpy=79>, 'Type_of_Travel': <tf.Tensor: shape=(), dtype=string, numpy=b'Business travel'>, 'Class': <tf.Tensor: shape=(), dtype=string, numpy=b'Business'>, 'Flight_Distance': <tf.Tensor: shape=(), dtype=int64, numpy=1861>, 'Seat_comfort': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'Departure_Arrival_time_convenient': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'Food_and_drink': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'Gate_location': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'Inflight_wifi_service': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'Inflight_entertainment': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'Online_support': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'Ease_of_Online_booking': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'On_board_service': <tf.Tensor: shape=()

In [20]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [21]:
from keras.layers import IntegerLookup
from keras.layers import Normalization
from keras.layers import StringLookup

def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [22]:
# Categorical features encoded as integers
confort = keras.Input(shape=(1,), name="Seat_comfort", dtype="int64")
da_time_conv = keras.Input(shape=(1,), name="Departure_Arrival_time_convenient", dtype="int64")
food = keras.Input(shape=(1,), name="Food_and_drink", dtype="int64")
gate = keras.Input(shape=(1,), name="Gate_location", dtype="int64")
wifi = keras.Input(shape=(1,), name="Inflight_wifi_service", dtype="int64")
entertainement = keras.Input(shape=(1,), name="Inflight_entertainment", dtype="int64")
support = keras.Input(shape=(1,), name="Online_support", dtype="int64")
booking = keras.Input(shape=(1,), name="Ease_of_Online_booking", dtype="int64")
service = keras.Input(shape=(1,), name="On_board_service", dtype="int64")
leg_service = keras.Input(shape=(1,), name="Leg_room_service", dtype="int64")
clean = keras.Input(shape=(1,), name="Cleanliness", dtype="int64")
baggage = keras.Input(shape=(1,), name="Baggage_handling", dtype="int64")
check = keras.Input(shape=(1,), name="Checkin_service", dtype="int64")
board = keras.Input(shape=(1,), name="Online_boarding", dtype="int64")

# Categorical feature encoded as string
type_travel = keras.Input(shape=(1,), name="Type_of_Travel", dtype="string")
customer_type = keras.Input(shape=(1,), name="Customer_Type", dtype="string")
classe = keras.Input(shape=(1,), name="Class", dtype="string")
sex = keras.Input(shape=(1,), name="Gender", dtype="string")

# Numerical features
age = keras.Input(shape=(1,), name="Age")
distance = keras.Input(shape=(1,), name="Flight_Distance")
delay_dep = keras.Input(shape=(1,), name="Departure_Delay_in_Minutes")
delay_arr = keras.Input(shape=(1,), name="Arrival_Delay_in_Minutes")

all_inputs = [
    type_travel,
    customer_type,
    classe,
    sex,
    age,
    distance,
    confort,
    da_time_conv,
    food,
    gate,
    wifi,
    entertainement,
    support,
    booking,
    service,
    leg_service,
    baggage,
    check,
    clean,
    board,
    delay_dep,
    delay_arr,
]

# Integer categorical features
confort_encoded = encode_categorical_feature(confort, "Seat_comfort", train_ds, False)
da_time_conv_encoded = encode_categorical_feature(da_time_conv, "Departure_Arrival_time_convenient", train_ds, False)
food_encoded = encode_categorical_feature(food, "Food_and_drink", train_ds, False)
gate_encoded = encode_categorical_feature(gate, "Gate_location", train_ds, False)
support_encoded = encode_categorical_feature(support, "Online_support", train_ds, False)
entertainement_encoded = encode_categorical_feature(entertainement, "Inflight_entertainment", train_ds, False)
wifi_encoded = encode_categorical_feature(wifi, "Inflight_wifi_service", train_ds, False)
booking_encoded = encode_categorical_feature(booking, "Ease_of_Online_booking", train_ds, False)
service_encoded = encode_categorical_feature(service, "On_board_service", train_ds, False)
leg_service_encoded = encode_categorical_feature(leg_service, "Leg_room_service", train_ds, False)
baggage_encoded = encode_categorical_feature(baggage, "Baggage_handling", train_ds, False)
check_encoded = encode_categorical_feature(check, "Checkin_service", train_ds, False)
clean_encoded = encode_categorical_feature(clean, "Cleanliness", train_ds, False)
board_encoded = encode_categorical_feature(board, "Online_boarding", train_ds, False)

# String categorical features
type_travel_encoded = encode_categorical_feature(type_travel, "Type_of_Travel", train_ds, True)
customer_type_encoded = encode_categorical_feature(customer_type, "Customer_Type", train_ds, True)
classe_encoded = encode_categorical_feature(classe, "Class", train_ds, True)
sex_encoded = encode_categorical_feature(sex, "Gender", train_ds, True)

# Numerical features
age_encoded = encode_numerical_feature(age, "Age", train_ds)
distance_encoded = encode_numerical_feature(distance, "Flight_Distance", train_ds)
delay_dep_encoded = encode_numerical_feature(delay_dep, "Departure_Delay_in_Minutes", train_ds)
delay_arr_encoded = encode_numerical_feature(delay_arr, "Arrival_Delay_in_Minutes", train_ds)

all_features = layers.concatenate(
    [
        type_travel_encoded,
        customer_type_encoded,
        classe_encoded,
        sex_encoded,
        age_encoded,
        distance_encoded,
        confort_encoded,
        da_time_conv_encoded,
        food_encoded,
        gate_encoded,
        train_ds,
        wifi_encoded,
        entertainement_encoded,
        support_encoded,
        booking_encoded,
        service_encoded,
        leg_service_encoded,
        baggage_encoded,
        check_encoded,
        clean_encoded,
        board_encoded,
        delay_dep_encoded,
        delay_arr_encoded,
    ]
)
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

2023-04-06 15:06:52.291961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64 and shape [103590]
	 [[{{node Placeholder/_7}}]]
2023-04-06 15:06:52.292974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [103590]
	 [[{{node Placeholder/_12}}]]
2023-04-06 15:06:56.109805: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1

ValueError: as_list() is not defined on an unknown TensorShape.